In [34]:
# 1. Импортируем библиотеки
import pandas as pd 
import re

In [35]:
# 2. Считываем датафрейм.
path = ("_data.csv")
df = pd.read_csv(path)

In [36]:
# 3. Устнавливаем опцию отображения всех столбцов.
pd.set_option('display.max_columns', None)

In [37]:
# 4. Оставляем в датафрейме объявление только по Москве.
df = df[df["Адрес"].str.contains("Москва", case=False)]

In [38]:
# 5. Ищем дубликаты объявлений по описанию.
duplicate_rows = df[df.duplicated(subset='Описание')]
len(duplicate_rows)

281

In [39]:
# 6. Удаляем дубликаты объявлений по описанию.
df = df.drop_duplicates(subset='Описание')
len(df)

19456

In [40]:
# 7. Удаляем лишние столбцы, которые не повлияют или незначительно повлияют на результат.
df.drop(["Unnamed: 0", "Тип", "Телефоны", "Описание", "Площадь комнат, м2", "Дополнительно", "Название ЖК", "Серия дома", "Ссылка на объявление"], inplace=True, axis=1)

In [41]:
# 8. Меняем название колонок.
rename_map = {'Количество комнат': "number_of_rooms", 'Метро': "metro", 'Адрес': "address", 'Площадь, м2': "apartment_area_m2", 
              'Дом': "house", 'Парковка': "parking_lot", 'Можно с детьми/животными': 'Children/pets allowed','Высота потолков, м': 'Ceiling_height',
              'Лифт': 'Elevator','Мусоропровод': 'Garbage_chute', 'Цена': 'price', 'Ремонт': 'decor', 'Балкон': 'balcony', 'Окна': 'windows', 
              'Санузел': 'bathroom'}

df.rename(columns=rename_map, inplace=True)

In [42]:
# 9. Заполняем пропущенные значения
df["number_of_rooms"].fillna("Апартаменты/свободная планировка", inplace=True)
df["metro"].fillna("Нет_данных", inplace=True)
df["parking_lot"].fillna("Отсутствует", inplace=True)

df['ceiling_height'] = df['Ceiling_height'].fillna(df['Ceiling_height'].mean())
df['elevator'] = df['Elevator'].fillna('Нет данных')
df['garbage_chute'] = df['Garbage_chute'].fillna('Нет_данных')
df['children/pets allowed'] = df['Children/pets allowed'].fillna('Нет_данных')

df['decor'].fillna('Нет_данных', inplace=True)
df['balcony'].fillna('Отсутствует',inplace=True)
df['windows'].fillna('Нет_данных',inplace=True)
df['bathroom'].fillna('Нет_данных',inplace=True)

In [43]:
df.to_csv("data.csv")